KNN scratch implementation

step 1) do basic pre processing,

step 2) Scratch implementaion of KNN,

step 3) KNN - SKlearn version

In [10]:
#import all libraries
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

1) Loading the dataset

In [11]:
!gdown 1ZdhRqYv-JizWV6DxO6C4R_k1kxPhmlF2
# Load CSV
df = pd.read_csv("multiclass.csv")

print("First 5 rows of dataset:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nShape of the dataset: ")
print(df.shape)


Downloading...
From: https://drive.google.com/uc?id=1ZdhRqYv-JizWV6DxO6C4R_k1kxPhmlF2
To: /content/multiclass.csv
100% 14.6k/14.6k [00:00<00:00, 35.6MB/s]
First 5 rows of dataset:
   Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen  class
0       3  12669  9656     7561     214              2674        1338      2
1       3   7057  9810     9568    1762              3293        1776      2
2       3   6353  8808     7684    2405              3516        7844      2
3       3  13265  1196     4221    6404               507        1788      1
4       3  22615  5410     7198    3915              1777        5185      1

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Region            440 non-null    int64
 1   Fresh             440 non-null    int64
 2   Milk              440 non-null    int64
 3   Grocery  

2) Handling null values

In [12]:
print("\nMissing Values:\n", df.isnull().sum())

# Drop rows/columns with too many missing values or fill with mean/mode
df = df.dropna()   # (or use df.fillna(method="ffill") depending on dataset)

print("\nAfter handling missing values:", df.shape)


Missing Values:
 Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
class               0
dtype: int64

After handling missing values: (440, 8)


3) Duplicate values

In [13]:
print("\nDuplicate Rows:", df.duplicated().sum())
df = df.drop_duplicates()

print("After removing duplicates:", df.shape)


Duplicate Rows: 0
After removing duplicates: (440, 8)


4) Encode Categorical values

In [14]:
cat_cols = df.select_dtypes(include=['object']).columns
print("\nCategorical Columns:", cat_cols)

# Label encode each categorical column
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

print("\nAfter Label Encoding:")
print(df.head())


Categorical Columns: Index([], dtype='object')

After Label Encoding:
   Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen  class
0       3  12669  9656     7561     214              2674        1338      2
1       3   7057  9810     9568    1762              3293        1776      2
2       3   6353  8808     7684    2405              3516        7844      2
3       3  13265  1196     4221    6404               507        1788      1
4       3  22615  5410     7198    3915              1777        5185      1


5) Feature scaling

In [15]:
X = df.iloc[:, :-1]   # all columns except last as features
y = df.iloc[:, -1]    # last column as target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nScaled Feature Sample:\n", X_scaled[:5])


Scaled Feature Sample:
 [[ 0.59066829  0.05293319  0.52356777 -0.04111489 -0.58936716 -0.04356873
  -0.06633906]
 [ 0.59066829 -0.39130197  0.54445767  0.17031835 -0.27013618  0.08640684
   0.08915105]
 [ 0.59066829 -0.44702926  0.40853771 -0.0281571  -0.13753572  0.13323164
   2.24329255]
 [ 0.59066829  0.10011141 -0.62401993 -0.3929769   0.6871443  -0.49858822
   0.09341105]
 [ 0.59066829  0.84023948 -0.05239645 -0.07935618  0.17385884 -0.23191782
   1.29934689]]


6) Splitting the data for training and testing

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTrain Shape:", X_train.shape, "Test Shape:", X_test.shape)


Train Shape: (352, 7) Test Shape: (88, 7)


=> KNN function

In [18]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def knn_predict(X_train, y_train, X_test, k=3):
    predictions = []
    # ensure numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    for x in X_test:
        # Compute distances
        distances = [np.sqrt(np.sum((x - x_train) ** 2)) for x_train in X_train]
        # Get indices of k nearest neighbors
        k_indices = np.argsort(distances)[:k]
        # Get labels
        k_neighbor_labels = [y_train[i] for i in k_indices]
        # Majority vote
        most_common = Counter(k_neighbor_labels).most_common(1)[0][0]
        predictions.append(most_common)
    return np.array(predictions)


y_pred_scratch = knn_predict(X_train, y_train, X_test, k=3)
print("\nScratch KNN Accuracy (function version):", accuracy_score(y_test, y_pred_scratch))


Scratch KNN Accuracy (function version): 0.9659090909090909


=> KNN using scikit learn


In [20]:
knn_sklearn = KNeighborsClassifier(n_neighbors=3)
knn_sklearn.fit(X_train, y_train)
y_pred_sklearn = knn_sklearn.predict(X_test)

print("Sklearn KNN Accuracy:", accuracy_score(y_test, y_pred_sklearn))

Sklearn KNN Accuracy: 0.9659090909090909
